<a href="https://colab.research.google.com/github/HannahMwende/Face-detection/blob/main/face_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Face detection system
This project implements a computer vision system designed to automatically detect a specific human face from images. By leveraging a pre-trained Haar Cascade classifier, the system efficiently identifies facial features within an image and draws bounding boxes around the detected faces. The system is built using OpenCV, a popular computer vision library, and aims to provide an intuitive solution for face recognition.

####Importing libraries

In [4]:
#Importing libraries
import os
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pickle

#### Loading data

In [6]:
#Connecting colab to gooogle drive
from google.colab import drive
drive.mount('/content/drive')

 #Printing contents of train_data folder
print(os.listdir("/content/drive/MyDrive/Face_detection/Data/Train_data"))

Mounted at /content/drive
['download.jpg', 'download (1).jpg', 'download (2).jpg']


In [7]:
#Loading a pre-trained Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


###Image pre-processing

In [8]:
input_folder = "/content/drive/MyDrive/Face_detection/Data/Train_data"
output_folder = "/content/drive/MyDrive/Face_detection/Data/Clean_data"

# Loop through each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # Filter for image files
        input_path = os.path.join(input_folder, filename)

        # Read the image
        image = cv2.imread(input_path)


        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

         # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


        # Draw a Bounding Box around the faces in the image
        for (x, y, w, h) in faces:
         face = gray_image[y:y+h, x:x+w]


          #Resize to a standard size (e.g., 100x100)
         face_resized = cv2.resize(face, (100, 100))

            # Normalize pixel values
         face_normalized = face_resized / 255.0  # Scale pixel values to [0, 1]

            # Save the processed face
         output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.jpg")
         cv2.imwrite(output_path, (face_normalized * 255).astype('uint8'))  # Save as uint8



Input: faces is a list of tuples returned by detectMultiScale. Each tuple (x, y, w, h) represents:
x: The x-coordinate of the top-left corner of the detected face.
y: The y-coordinate of the top-left corner of the detected face.
w: The width of the bounding box for the face.
h: The height of the bounding box for the face.


ray_frame[y:y+h, x:x+w]:

Extracts the region of interest (ROI) from gray_frame, which corresponds to the rectangle where a face is detected.
The slicing:
y:y+h: Rows in the image matrix that span the height of the face.
x:x+w: Columns in the image matrix that span the width of the face.
